In [ ]:
!pip install torchtext==0.6.0

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spi

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os,gc,re,time
import matplotlib.pyplot as plt
import seaborn as sns
import spacy,random
from collections import Counter
import warnings,unicodedata
warnings.simplefilter("ignore")


#setting device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Available device: {device}')

Available device: cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/transformer/')

In [ ]:
train_csv_path, val_csv_path, test_csv_path = 'data/train.csv','data/val.csv','data/test.csv'
train_data = pd.read_csv(train_csv_path)
val_data = pd.read_csv(val_csv_path)
test_data = pd.read_csv(test_csv_path)

print(train_data.shape, val_data.shape, test_data.shape)

(29000, 2) (1014, 2) (1000, 2)


In [ ]:
train_data.head()

,English,Ger
0,Two young White males are outside near many b...,Zwei junge weiße Männer sind im Freien in der ...
1,Several men in hard hats are operating a giant...,Mehrere Männer mit Schutzhelmen bedienen ein A...
2,A little girl climbing into a wooden playhouse.,Ein kleines Mädchen klettert in ein Spielhaus ...
3,A man in a blue shirt is standing on a ladder ...,Ein Mann in einem blauen Hemd steht auf einer ...
4,Two men are at the stove preparing food.,Zwei Männer stehen am Herd und bereiten Essen zu.


In [ ]:
val_data.head()

,English,Ger
0,A group of men are loading cotton onto a truck,Eine Gruppe von Männern lädt Baumwolle auf ein...
1,A man sleeping in a green room on a couch.,Ein Mann schläft in einem grünen Raum auf eine...
2,A boy wearing headphones sits on a woman's sho...,Ein Junge mit Kopfhörern sitzt auf den Schulte...
3,Two men setting up a blue ice fishing hut on a...,Zwei Männer bauen eine blaue Eisfischerhütte a...
4,A balding man wearing a red life jacket is sit...,Ein Mann mit beginnender Glatze der eine rote...


In [ ]:
test_data.head()

,English,Ger
0,A man in an orange hat starring at something.,Ein Mann mit einem orangefarbenen Hut der etw...
1,A Boston Terrier is running on lush green gras...,Ein Boston Terrier läuft über saftig-grünes Gr...
2,A girl in karate uniform breaking a stick with...,Ein Mädchen in einem Karateanzug bricht ein Br...
3,Five people wearing winter jackets and helmets...,Fünf Leute in Winterjacken und mit Helmen steh...
4,People are fixing the roof of a house.,Leute Reparieren das Dach eines Hauses.


In [ ]:
train_data.isnull().sum()

English    0
Ger        0
dtype: int64

In [ ]:
!pip install utils

In [ ]:
import re,unicodedata
import time,datetime

In [ ]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

2023-07-31 08:23:55.321152: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 27.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
2023-07-31 08:24:10.816406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
def unicodeToAscii(s):
    """
    convert to ascii string
    """
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

replace_dict = {"don't": "do not","shouldn't": "should not",
         "can't": "cannot", "I'm": "Iam", "I've": "I have",
         "haven't": "have not",
         "hasn't": "has not",
         "didn't": "did not"}

def clean_eng_text(text, replace_dict=replace_dict):
    """
    cleans english text
    Args:
        text: text to clean
        replace_dict: dict to replace words such as didn't with did not
    Returns:
        clean text
    """
    text = unicodeToAscii(text.lower().strip())
    # remove some special characters
    text = re.sub('[-?<>.!]', '', text)
    # remove extra spaces
    text = re.sub('\s+', ' ', text)
    text = ' '.join([replace_dict[word] if word in replace_dict.keys() else word for word in text.split()])
    text = [re.sub('[^A-Za-z]', '', word) for word in text.split()]
    return ' '.join(text).lower()

def clean_ger_text(text):
    """
    cleans german text
    Args:
        text: german text
    Returns:
        clean text
    """
    text = text.strip()
    text = re.sub("[.?]", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    return text

In [ ]:
train_data["English"] = train_data["English"].apply(lambda x: clean_eng_text(x))
train_data["Ger"] = train_data["Ger"].apply(lambda x: clean_ger_text(x))

val_data["English"] = val_data["English"].apply(lambda x: clean_eng_text(x))
val_data["Ger"] = val_data["Ger"].apply(lambda x: clean_ger_text(x))

test_data["English"] = test_data["English"].apply(lambda x: clean_eng_text(x))
test_data["Ger"] = test_data["Ger"].apply(lambda x: clean_ger_text(x))

In [ ]:
import en_core_web_sm, de_core_news_sm

class VocabGenerator():
    def __init__(self, corpus, min_frequency=2, tokenizer_lang="english"):
        """
        Args:
           corpus: list of text documents
           min_frequency: mimimum frequency by which a word should be included in corpus
           tokenizer_lang: language of tokenizer
        """
        self.stoi = {'<UNK>':0, '<PAD>':1, '<SOS>':2, '<EOS>':3}
        self.itos = {0:'<UNK>', 1:'<PAD>', 2:'<SOS>', 3:'<EOS>'}
        self.min_freq = min_frequency

        #tokenizer
        self.lang = tokenizer_lang
        if tokenizer_lang == "english":
            self.tokenizer = en_core_web_sm.load()
        else:
            self.tokenizer = de_core_news_sm.load()

        self.build_vocabulary(corpus)

    def __len__(self):
        """
        Returns length of the vocabulary
        """
        return len(self.itos)

    def counter(self,text):
        """
        counts the frequency of words in corpus
        Args:
            text : text for splitting
        """

        out = {}
        text = text.split()
        for word in text:
            if word in out:
                out[word] += 1
            else:
                out[word] = 1
        return out

    def build_vocabulary(self,corpus):
        """
        function to build vocabulary from corpus.It updates stoi and itos dictionaries
        Args:
            corpus: list of documents
        """
        idx = 4
        frequency_dict = self.counter(corpus)
        for word,freq in frequency_dict.items():
          if freq >= self.min_freq:
              self.stoi[word] = idx
              self.itos[idx] = word
              idx += 1

    def generate_numeric_tokens(self,text):
        """
        generates token from text
        Args:
            text: input text
        """

        tokenized_out = [token.text for token in self.tokenizer(text)]
        out = [self.stoi[token] if token in self.stoi.keys() else self.stoi["<UNK>"] for token in tokenized_out]
        return out

    def add_eos_sos(self,numeric_list):
        """
        add EOS and SOS token to tokens already generated
        Args:
           numeric_list: list of integers generated from token
        """
        numeric_list = [self.stoi['<SOS>']] + numeric_list + [self.stoi['<EOS>']]
        return numeric_list

    def pad_sequence(self,numeric_list,max_seq_length):
        """
        function which do padding and truncation of integer list of tokens
        Args:
            numeric_list: integer list of tokens
            max_seq_length: length to which sequence is to be padded
        """
        if len(numeric_list) < max_seq_length:
            no_zeros = max_seq_length - len(numeric_list)
            numeric_list = numeric_list + [self.stoi['<PAD>'] for i in range(no_zeros)]
        else:
            numeric_list = numeric_list[:max_seq_length]

        return numeric_list

In [ ]:
#creating corpus
corpus_eng = ' '.join(list(train_data['English'].values))
vocab_eng = VocabGenerator(corpus=corpus_eng, min_frequency=2,tokenizer_lang="english")

corpus_ger = ' '.join(list(train_data['Ger'].values))
vocab_ger = VocabGenerator(corpus=corpus_ger, min_frequency=2,tokenizer_lang="germen")

In [ ]:
print(f"len(SRC): {len(vocab_ger)}")
print(f"len(TRG): {len(vocab_eng)}")

len(SRC): 7971
len(TRG): 5918


In [ ]:
print(vocab_eng.stoi["<PAD>"])  # 패딩(padding): 1
print(vocab_eng.stoi["<SOS>"])        #  : 2
print(vocab_eng.stoi["<EOS>"])       # : 3
print(vocab_eng.stoi["water"])
print(vocab_eng.stoi["world"])

1
2
3
264
4070


In [ ]:
# print(vocab_ger.stoi)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class LoadTextDataset(Dataset):
    def __init__(self, df, vocab_input, vocab_target, input_seq_length=None, target_seq_length=None, padding=False):
        """
        Loads data from dataframe
        Args:
            df : dataframe
            vocab_input: dictionary of input text vocabulary
            vocab_target: dictionary of target text vocabulary
            input_seq_length: length of input sequence without SOS and EOS token
            target_seq_length: length of target sequence without SOS and EOS token
            padding: Do we need padding or not
        """
        self.df = df
        self.padding = padding
        self.input_text_values = df['Ger'].values
        self.target_text_values = df['English'].values
        self.vocab_input = vocab_input
        self.vocab_target = vocab_target
        self.input_seq_length = input_seq_length
        self.target_seq_length = target_seq_length

    def __len__(self):
        """
        returns length of dataframe for dataloader
        """
        return len(self.df)

    def __getitem__(self, idx):
        """
        Args:
            idx: index
        """
        input_text, target_text = self.input_text_values[idx], self.target_text_values[idx]

        # preprocessing text
        input_text, target_text = clean_ger_text(input_text), clean_eng_text(target_text)

        text_numeric_input = self.vocab_input.generate_numeric_tokens(input_text)
        text_numeric_target = self.vocab_target.generate_numeric_tokens(target_text)
        text_numeric_input = self.vocab_input.add_eos_sos(text_numeric_input)
        text_numeric_target = self.vocab_target.add_eos_sos(text_numeric_target)

        if self.padding:
            text_numeric_input = self.vocab_input.pad_sequence(text_numeric_input, self.input_seq_length + 2)
            text_numeric_target = self.vocab_target.pad_sequence(text_numeric_target, self.target_seq_length + 2)

        input_seq, target_seq = torch.Tensor(text_numeric_input).long(), torch.Tensor(text_numeric_target).long()

        return input_seq, target_seq

In [ ]:
batch_size = 128

df_train = train_data.drop(train_data.tail((train_data.shape[0]%batch_size)).index)
df_val = val_data.drop(val_data.tail((val_data.shape[0]%batch_size)).index)
df_test = test_data.drop(test_data.tail((test_data.shape[0]%batch_size)).index)

print(df_train.shape,df_val.shape,df_test.shape)

(28928, 2) (896, 2) (896, 2)


In [ ]:
!pip install utils

In [ ]:
!ls utils

data_loader.py	__pycache__  utils.py  vocab_generator.py


In [ ]:
!cd /content/drive/MyDrive/Colab Notebooks/transformer/utils

/bin/bash: line 1: cd: too many arguments


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/transformer/utils')

In [ ]:
train_data = LoadTextDataset(df_train, vocab_ger, vocab_eng, input_seq_length=23, target_seq_length=23, padding=True)
val_data = LoadTextDataset(df_val, vocab_ger, vocab_eng, input_seq_length=23, target_seq_length=23, padding=True)
test_data = LoadTextDataset(df_test, vocab_ger, vocab_eng, input_seq_length=23, target_seq_length=23, padding=True)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size=batch_size)
valid_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

for data,target in train_loader:
    print(data.shape,target.shape)
    break

for data,target in valid_loader:
    print(data.shape,target.shape)
    break

torch.Size([128, 25]) torch.Size([128, 25])
torch.Size([128, 25]) torch.Size([128, 25])


In [ ]:
print(f'Train data:')
for input_seq,target_seq in train_data:
    print(input_seq)
    print(target_seq)
    break

print('-'*100)
print('Validation data:')
for input_seq,target_seq in val_data:
    print(input_seq)
    print(target_seq)
    break

print('-'*100)
print('Test data:')
for input_seq,target_seq in test_data:
    print(input_seq)
    print(target_seq)
    break

Train data:
tensor([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  3,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1])
tensor([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12,  3,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1])
----------------------------------------------------------------------------------------------------
Validation data:
tensor([   2,   66,  125,   84,  395, 1848,    0,   33,   54,  991,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1])
tensor([   2,   19,  240,   72,   14,    8, 1048, 1250, 1502,   19,  669,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1])
----------------------------------------------------------------------------------------------------
Test data:
tensor([  2,  21,  28,  17,  29,  97, 200,  12, 174,   0,   3,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])
tensor([  2,  19,  29,  15, 200,

In [ ]:
# @title Embedding


In [ ]:
import math

class Embedding_Layer(nn.Module):

    def __init__(self, vocab_size, emb_size):
        super(Embedding_Layer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [ ]:
# @title Positional encoding


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,embed_model_dim, max_length=200):
        super(PositionalEncoding, self).__init__()

        self.embed_dim = embed_model_dim
        pe = torch.zeros(max_length, self.embed_dim)
        for pos in range(max_length):
            for i in range(0, self.embed_dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/self.embed_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/self.embed_dim)))

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):

        seq_len = x.size(1)
        x = x + torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)
        return x

In [ ]:
# @title Scaled dot-product


In [ ]:
import torch.nn.functional as F

class ScaledDotProduct_Attention(nn.Module):

    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, q, k, v, mask=None):

        attn = torch.matmul(q / self.temperature, k.transpose(2, 3))

        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)

        attn = self.dropout(F.softmax(attn, dim=-1))

        output = torch.matmul(attn, v)

        return output, attn

In [ ]:
# @title Multihead attention
